<a href="https://colab.research.google.com/github/michal-uszko/pad_cwiczenia/blob/main/wpbd_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [3]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

!pip install -q findspark

import findspark
 
findspark.init()

findspark.find()# sprawdzenie gdzie znajduje się instalka

'/content/spark-3.3.1-bin-hadoop3'

Zadanie 1 - total spent by customer

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# tworzymy SparkSession zamiast context
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# getOrCreate tworzy nową sesję albo podłącza się do istniejącej

def mapper(line):
    fields = line.split(',')
    return Row(id_client=int(fields[0]), id_item=int(fields[1]), order_value=float(fields[2]))

# dane nie mają nagłówka, dlatego korzystamy z contextu żeby załadować dane
lines = spark.sparkContext.textFile("/content/drive/MyDrive/WPBD/lab7/materialy/praca domowa/customer-orders.csv")
# nasza funkcja przekonwertuje nasze dane na wiersze i pola z nazwami kolumn i typami danych
people = lines.map(mapper)

# tworzymy dataframe
# cacheujemy ponieważchcemy wysyłać wiele zapytań i trzeba to jakoś przechować w pamięci
schemaPeople = spark.createDataFrame(people).cache()
# tworzymy widok jak w bazie danych aby wysyłać zapytania, widok people działa jak tabela
schemaPeople.createOrReplaceTempView("customer_orders")

schemaPeople.groupBy("id_client").sum("order_value").withColumnRenamed("sum(order_value)", "sum_order_value").show(100)

spark.stop()

+---------+------------------+
|id_client|   sum_order_value|
+---------+------------------+
|       29| 5032.530000000001|
|       26|            5250.4|
|       65|           5140.35|
|       54| 6065.390000000001|
|       19|           5059.43|
|        0| 5524.949999999999|
|       22| 5019.449999999999|
|        7| 4755.069999999999|
|       77|           4327.73|
|       34|            5330.8|
|       50|4517.2699999999995|
|       94| 4475.570000000001|
|       57|            4628.4|
|       43|           5368.83|
|       32|           5496.05|
|       84|4652.9400000000005|
|       31| 4765.049999999999|
|       98| 4297.259999999999|
|       39| 6193.110000000001|
|       25| 5057.610000000001|
|       95| 4876.839999999998|
|       71| 5995.660000000002|
|        6| 5397.879999999999|
|       68| 6375.450000000001|
|       72| 5337.439999999999|
|       87|            5206.4|
|       58|           5437.73|
|        9|           5322.65|
|       27|           4915.89|
|       

Zadanie 2 - sorted total spent by customer

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# tworzymy SparkSession zamiast context
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# getOrCreate tworzy nową sesję albo podłącza się do istniejącej

def mapper(line):
    fields = line.split(',')
    return Row(id_client=int(fields[0]), id_item=int(fields[1]), order_value=float(fields[2]))

# dane nie mają nagłówka, dlatego korzystamy z contextu żeby załadować dane
lines = spark.sparkContext.textFile("/content/drive/MyDrive/WPBD/lab7/materialy/praca domowa/customer-orders.csv")
# nasza funkcja przekonwertuje nasze dane na wiersze i pola z nazwami kolumn i typami danych
people = lines.map(mapper)

# tworzymy dataframe
# cacheujemy ponieważchcemy wysyłać wiele zapytań i trzeba to jakoś przechować w pamięci
schemaPeople = spark.createDataFrame(people).cache()
# tworzymy widok jak w bazie danych aby wysyłać zapytania, widok people działa jak tabela
schemaPeople.createOrReplaceTempView("customer_orders")

schemaPeople.groupBy("id_client").sum("order_value").orderBy("sum(order_value)").withColumnRenamed("sum(order_value)", "sum_order_value").show(100)

spark.stop()

+---------+------------------+
|id_client|   sum_order_value|
+---------+------------------+
|       45|3309.3799999999997|
|       79|3790.5699999999997|
|       96|3924.2300000000005|
|       23|           4042.65|
|       99|           4172.29|
|       75|            4178.5|
|       36| 4278.049999999999|
|       98| 4297.259999999999|
|       47| 4316.299999999998|
|       77|           4327.73|
|       13| 4367.619999999999|
|       48|           4384.33|
|       49|            4394.6|
|       94| 4475.570000000001|
|       67|           4505.79|
|       50|4517.2699999999995|
|       78|           4524.51|
|        5|           4561.07|
|       57|            4628.4|
|       83|            4635.8|
|       91|           4642.26|
|       74| 4647.130000000001|
|       84|4652.9400000000005|
|        3|           4659.63|
|       12| 4664.589999999999|
|       66|           4681.92|
|       56|           4701.02|
|       21|           4707.41|
|       80|           4727.86|
|       